# Анализ базы данных сервиса для чтения книг по подписке

## Цели исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В
свободное время жители городов больше не выходят на улицу, не посещают кафе
и торговые центры. Зато стало больше времени для книг. Это заметили
стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг
по подписке. Ваша первая задача как аналитика — проанализировать базу
данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские
обзоры книг. Эти данные помогут сформулировать ценностное предложение для
нового продукта.

__Задачи:__
1. Посчитать, сколько книг вышло после 1 января 2000 года.
2. Для каждой книги посчитать количество обзоров и среднюю оценку.
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц.
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками.
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

***Описание данных:***
Таблица `books`
Содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.


Таблица `authors`
Содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица `publishers`
Содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;


Таблица `ratings`
Содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews`
Содержит данные о пользовательских обзорах на книги:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора.

## Библиотеки, функции, подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

from sqlalchemy import create_engine

In [2]:
#функция для выполнения sql-запроса
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

In [4]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

In [5]:
for i in tables:
    print('\033[1m','Таблица', i, ':')
    query = '''
    SELECT *
    FROM {}
    LIMIT 5
    ''' .format(i)
    display(sql_result(query))
    print()

 Таблица books :


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



 Таблица authors :


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



 Таблица publishers :


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



 Таблица ratings :


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



 Таблица reviews :


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


In [6]:
for i in tables:
    print('\033[1m','Таблица', i, ':','\033[0m')
    query = '''
    SELECT *
    FROM {}
    ''' .format(i)
    display(sql_result(query).info())
    print()

 Таблица books : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None


 Таблица authors : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None


 Таблица publishers : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None


 Таблица ratings : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None


 Таблица reviews : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

### Вывод

Произведен предпосмотр данных, предобработка в рамках проекта не производилась:
- пропуски в данных не обнаружены.
- предварительно данных достаточно для проведения анализа.

## Задачи:

### Книги, которые вышли после 1 января 2000 года

In [7]:
query = '''
SELECT count(book_id) books_count
FROM books
WHERE publication_date >= '2000-01-01'
''' 
sql_result(query) 

,books_count
0,821


#### Вывод

С 1 января 2000 года вышла 821 книга.

### Количество обзоров и среднюю оценку для каждой книги

In [8]:
query = '''
WITH 
review as (
          SELECT book_id,
                 count(review_id) reviews_cnt
          FROM reviews
          GROUP BY book_id),
rating as (
            SELECT book_id,
                   avg(rating) as avg_rating
            FROM ratings
            GROUP BY book_id)
SELECT title,
       rt.avg_rating,
       rv.reviews_cnt
FROM books b
LEFT JOIN rating rt ON b.book_id=rt.book_id
LEFT JOIN review rv ON b.book_id=rv.book_id
ORDER BY  3 desc, 2 desc
LIMIT 15
''' 
sql_result(query)

,title,avg_rating,reviews_cnt
0,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,Essential Tales and Poems,4.00,NaN
2,Leonardo's Notebooks,4.00,NaN
3,Disney's Beauty and the Beast (A Little Golden Book),4.00,NaN
4,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,NaN
5,The Natural Way to Draw,3.00,NaN
6,Twilight (Twilight #1),3.66,7.00
7,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),4.41,6.00
8,Harry Potter and the Chamber of Secrets (Harry Potter #2),4.29,6.00
9,The Book Thief,4.26,6.00


####  Проверка

In [9]:
# Количество ревью в базовой таблице
query = '''
SELECT count(review_id) as reviews_orig
FROM reviews 
    
''' 
print('Количество ревью в изначальной таблице:')
display(sql_result(query))

# Количество ревью в агрегированной таблице
query = '''

WITH 
review as (
          SELECT book_id,
                 count(review_id) reviews_cnt
          FROM reviews
          GROUP BY book_id),
rating as (
            SELECT book_id,
                   avg(rating) as avg_rating
            FROM ratings
            GROUP BY book_id)
SELECT sum(rv.reviews_cnt) as reviews_aggr
FROM books b
LEFT JOIN rating rt ON b.book_id=rt.book_id
LEFT JOIN review rv ON b.book_id=rv.book_id

''' 
print('Количество ревью в агрегированной таблице:')
sql_result(query)

Количество ревью в изначальной таблице:


,reviews_orig
0,2793


Количество ревью в агрегированной таблице:


,reviews_aggr
0,"2,793.00"


In [10]:
# Сумма средних рэйтингов книг в базовой таблице
query = '''
SELECT sum(avg_rating) as avg_rating_orig
FROM (SELECT book_id,
            avg(rating) as avg_rating
     FROM ratings
     GROUP BY book_id) foo
    
''' 
print('Сумма средних рэйтингов книг в изначальной таблице:')
display(sql_result(query))

# Сумма средних рэйтингов книг в агрегированной таблице
query = '''

WITH 
review as (
          SELECT book_id,
                 count(review_id) reviews_cnt
          FROM reviews
          GROUP BY book_id),
rating as (
            SELECT book_id,
                   avg(rating) as avg_rating
            FROM ratings
            GROUP BY book_id)

SELECT sum(rt.avg_rating) avg_rating_aggr
FROM books b
LEFT JOIN rating rt ON b.book_id=rt.book_id
LEFT JOIN review rv ON b.book_id=rv.book_id

''' 
print('Сумма средних рэйтингов книг в агрегированной таблице:')
sql_result(query)

Сумма средних рэйтингов книг в изначальной таблице:


,avg_rating_orig
0,"3,898.97"


Сумма средних рэйтингов книг в агрегированной таблице:


,avg_rating_aggr
0,"3,898.97"


#### Вывод

1. Наибольшее количество обзоров у
    * `Twilight (Twilight №1)` (Сумерки, 1 книга) - 7 шт.
    * `Harry Potter`(Гарри Поттер, 2 и 3 книги) - 6 шт.
    
2. У шести книг из тысячи нет ни одного отзыва.
3. У более популярных книг средняя оценка зрителей как правило ниже, чем у не популярных книг.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [11]:
query = '''
SELECT publisher
FROM publishers
WHERE publisher_id = (
                      SELECT publisher_id
                      FROM books WHERE num_pages > 50
                      GROUP BY 1
                      ORDER BY count(book_id) DESC
                      LIMIT 1)
''' 
sql_result(query) 

,publisher
0,Penguin Books


#### Вывод

Наибольшее число книг толще 50 страниц выпустило издательство `Penguin Books`.

### Автор с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)

In [12]:
query = '''
SELECT author,
       avg(avg_rating) AS max_rating
FROM
  (SELECT avg(r.rating) AS avg_rating,
          a.author
   FROM ratings AS r
   JOIN books b ON r.book_id = b.book_id
   JOIN authors a ON b.author_id = a.author_id
   GROUP BY a.author,
            r.book_id
   HAVING count(r.rating_id) >= 50) AS top_author
GROUP BY author
ORDER BY max_rating DESC
LIMIT 5

''' 
sql_result(query) 

,author,max_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


#### Вывод

Самая высокая средняя оценка книг у `J.K. Rowling/Mary GrandPré	` - автора и иллюстратора серии книг о Гарри Поттере.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = '''
WITH top_user AS
  (SELECT username
   FROM ratings
   GROUP BY username
   HAVING count(rating_id) > 50),
     avg_review AS
  (SELECT username,
          count(review_id) AS review_cnt
   FROM reviews
   GROUP BY username)
SELECT avg(a.review_cnt) AS avg_review
FROM top_user AS top
JOIN avg_review AS a ON top.username = a.username

''' 
sql_result(query) 

,avg_review
0,24.33


#### Вывод

У пользователей, которые поставили более 50 оценок, в среднем 24 обзора. Таким образом, активные пользователи практически в 50% случаев ставят не только оценку, но и оставляют отзыв на книгу. 

## Вывод

В ходе анализа базы данных сервиса для чтения книг по подписке получена следующая информация:
1. С 1 января 2000 года вышла 821 книга.
2. Для каждой книги подсчитаны количество обзоров и средняя оценка:
     - Наибольшее количество обзоров у
        * `Twilight (Twilight #1)` (Сумерки, 1 книга) - 7 шт.;
        * `Harry Potter`(Гарри Поттер, 2 и 3 книги) - 6 шт.
     - У шести книг из тысячи нет ни одного отзыва.
     - У более популярных книг средняя оценка зрителей как правило ниже, чем у не популярных книг.
3. Наибольшее число книг толще 50 страниц выпустило издательство `Penguin Books`.
4. Самая высокая средняя оценка книг у `J.K. Rowling/Mary GrandPré	` - автора и иллюстратора серии книг о Гарри Поттере.
5. У пользователей, которые поставили более 50 оценок, в среднем 24 обзора. Таким образом, активные пользователи практически в 50% случаев ставят не только оценку, но и оставляют отзыв на книгу.